## Define the problem

## Find a dataset

dataset: https://www.kaggle.com/datasets/goyaladi/twitter-bot-detection-dataset?select=bot_detection_data.csv

In [ ]:
%pip install pandas numpy matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Citation
Dataset: [Twitter Bot Detection Dataset](https://www.kaggle.com/datasets/goyaladi/twitter-bot-detection-dataset)  
Author: Goyal Aditya  
Platform: Kaggle  
Accessed: 2025-01-19

Ensure kaggle.json is in the location ~/.kaggle/kaggle.json to use the API.

In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("goyaladi/twitter-bot-detection-dataset")

print("Path to dataset files:", path)

KaggleApiHTTPError: 404 Client Error.

Resource not found at URL: https://www.kaggle.com/datasets/goyaladi/twitter-bot-detection-dataset
Please make sure you specified the correct resource identifiers.